3. Language model inference methods: properties, differences, cases of usage (without formulas, at least 2)  
1 Свойства
Большая часть из методов использует механизм Attention и LSTM для запоминания длинных цепочек из слов. Главная цель таким образом задать последовательность слов, чтобы из нее можно было вытянуть главную мысль

2 Различия
Различия могут быть в тренировке в частности в сравнивании эмбеддингов, в попытке извлечь главную мысль. Также есть методы основанные не на РНН а на Трансформерах
3 Использование
Q&A, paraphrasing

# Exam

Develop a model for predicting review rating.  
**Multiclass classification into 5 classes**  
Score: **F1 with macro averaging**  
You are forbidden to use test dataset for any kind of training.  
Remember proper training pipeline.  
If you are not using default params in the models, you have to use some validation scheme to justify them. 

Use `random_state` or `seed` params - your experiment must be reprodusible.


### 1 baseline = 0.51
### 2 baseline = 0.53


In [2]:
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('punkt')
!pip install pymorphy2
import pymorphy2
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_train["review"] = df_train["review"] + " " + df_train["title"]
df_train = df_train.drop(columns=["title"])
df_test["review"] = df_test["review"] + " " + df_test["title"]
df_test = df_test.drop(columns=["title"])

In [0]:
morph = pymorphy2.MorphAnalyzer()
stopWords = stopwords.words('russian')
def cleaning(text):
    tokenized = []
    words = word_tokenize(text)
    for word in words:
        p = morph.parse(word)[0]
        tokenized.append(p.normal_form)
    tokenized = [token for token in tokenized if token not in stopWords\
                and token != " " \
                and token != "—" \
                and token != "«" \
                and token != "»" \
                and token != ".." \
                and token.strip() not in punctuation]
    return tokenized
df_train_X = df_train['review'].apply(cleaning)
df_test_X = df_test['review'].apply(cleaning)

In [0]:
train_X = df_train_X.tolist()
train_y = df_train["target"].tolist()
test_X = df_test_X.tolist()
test_y = df_test["target"].tolist()

In [63]:
from gensim.models import Word2Vec
from gensim.models import Phrases
bigram_transformer = Phrases(train_X)
w2v = Word2Vec(min_count=1, window=3, size=100)
w2v.build_vocab(bigram_transformer[train_X])
w2v.train(train_X, total_examples=w2v.corpus_count, epochs=10)



/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


(23243585, 32293390)

In [0]:
w2v.init_sims(replace=True)

In [100]:
w2v.wv.vocab.keys()

dict_keys(['the', 'staff', 'was', 'very', 'friendly', 'breakfast', 'nice', 'extremely', 'comfortable', 'beds', 'walker', 'gem', 'excellent', 'service', 'approachable', 'and', 'professional', 'i', 'stayed_here', 'on', 'business', 'dining', 'gallery', 'certainly', 'convenient', 'fitness_center', 'could', 'be_improved', 'but', 'it', 'has', 'basic', 'cardio', 'equipment', 'needed', 'would', 'stay', 'again', 'really', 'a', 'top_notch', 'place', 'to', 'spend', 'day', 'at', 'beginning', 'or', 'end', 'of', 'honduras', 'trip', 'is', 'helpful', 'efficient', 'you', 'have', 'questions', 'they', "'ll", 'find', 'answers', 'make', 'calls', 'for', "'s", 'walking_distance', 'from', 'mall', 'lots_of', 'good', 'restaurants', 'though', 'advisable', 'take', 'taxi', 'night', 'breakfast_buffet', '...', 'location', 'warm', 'little_noisy', 'there', 'false', 'fire_alarm', 'midnight', 'no', 'reason', 'given', 'hotel', 'had', 'too', 'many', 'animals', "i_'m", 'allergic', 'pets', 'although', 'did', 'receive', 'pet

In [0]:
def sent2vec (sent):
  sentvec = np.zeros(100)
  for word in sent:
    if word in w2v.wv.vocab.keys():
      sentvec += w2v.wv[word]
  sentvec = sentvec/len(sent)
  return sentvec

In [0]:
X_train = df_train_X.apply(sent2vec).tolist()
X_test = df_test_X.apply(sent2vec).tolist()

In [110]:
neighbourscos = KNeighborsClassifier(n_neighbors=8, metric="cosine")
neighbourscos.fit(X_train, train_y)
numb_of_neighbours = np.arange(1,50)
gridparam = dict(n_neighbors = numb_of_neighbours)
knncos = GridSearchCV(neighbourscos, gridparam, scoring = 'f1_macro')
knncos.fit(X_train, train_y)
print(knncos.best_params_['n_neighbors'])

33


In [111]:
print('f1 on validation: ', knncos.best_score_)

f1 on validation:  0.4387546280808028


In [115]:
from sklearn.metrics import f1_score
neighbourscos = KNeighborsClassifier(n_neighbors=knncos.best_params_['n_neighbors'], metric="euclidean")
neighbourscos.fit(X_train, train_y)
neighbourscos_test = f1_score(test_y, neighbourscos.predict(X_test), average = 'macro')
print('knn cosine test score: ', neighbourscos_test)

knn cosine test score:  0.43099439945424506


In [4]:
!pip install allennlp
import allennlp

In [0]:
elmo_df_train = df_train['review'].tolist()
elmo_df_test = df_test['review'].tolist()
train_y = df_train["target"].tolist()
test_y = df_test["target"].tolist()

In [0]:
from allennlp.modules.elmo import Elmo, batch_to_ids

options_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_options.json"
weight_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5"


elmo = Elmo(options_file, weight_file, 2, dropout=0)


In [0]:
elmo_train = batch_to_ids(elmo_df_train)
elmo_test = batch_to_ids(elmo_df_test)

embeddings_train = elmo(elmo_train)
embeddings_test = elmo(elmo_test)


In [0]:
neighbourscos = KNeighborsClassifier(n_neighbors=8, metric="cosine")
neighbourscos.fit(embeddings_train, train_y)
numb_of_neighbours = np.arange(1,50)
gridparam = dict(n_neighbors = numb_of_neighbours)
knncos = GridSearchCV(neighbourscos, gridparam, scoring = 'f1_macro')
knncos.fit(embeddings_train, train_y)
print(knncos.best_params_['n_neighbors'])

In [0]:
from sklearn.metrics import f1_score
neighbourscos = KNeighborsClassifier(n_neighbors=knncos.best_params_['n_neighbors'], metric="euclidean")
neighbourscos.fit(embeddings_train, train_y)
neighbourscos_test = f1_score(test_y, neighbourscos.predict(embeddings_test), average = 'macro')
print('knn cosine test score: ', neighbourscos_test)